## LLM 도메인 예측
### 타겟별 Noisy dataset을 예시로 넣고, 주제를 예측하라고 입력

In [5]:
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [6]:
# 사용할 LLM 모델 설정
llm = Ollama(model="gemma2")

/tmp/ipykernel_589628/3434335749.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma2")


In [11]:
import pandas as pd
from typing import Dict, List
import json
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     '''당신은 손상된 기사 제목들을 종합적으로 분석하여 전체 기사들의 공통된 도메인을 예측하는 어시스턴트입니다.
     주어지는 모든 기사 제목들을 함께 고려하여, 이 기사들이 속한 하나의 도메인을 판단하세요.
     도메인은 한국어이며, 한 단어로 표현해주세요.
     모든 기사 제목들에서 발견되는 주요 키워드들을 추출하세요.
     
     도메인 예시:
     - 정치
     - 북한
     - 경제
     - 산업
     - 사회
     - 전국
     - 세계
     - 문화
     - 건강
     - 연예
     - 스포츠
     
     다음 정보를 제공하세요:
     1. 모든 기사들에서 추출된 주요 키워드들을 하나의 리스트로
     2. 기사들의 공통된 도메인
     3. 도메인 예측 근거
     '''),
    ("human", 
     '''다음 기사 제목들을 분석하여 하나의 도메인으로 분류해주세요:
     
     기사들: {examples}
     
     {format_instructions}
     ''')
])

def create_unified_domain_classifier(llm):
    """통합 도메인 분류기 체인 생성"""
    chain = prompt | llm | parser
    
    def classify_domain(examples: List[str]) -> Dict:
        """여러 기사 제목들을 하나의 도메인으로 분류"""
        response = chain.invoke({
            "examples": "\n".join(examples),
            "format_instructions": """출력은 다음 JSON 형식이어야 합니다:
            {
                "keywords": ["키워드1", "키워드2", ...],
                "domain": "예측된_도메인",
                "reasoning": "도메인 예측 근거"
            }"""
        })
        return response
    
    return classify_domain

In [12]:
classifier = create_domain_classifier(llm)

examples = [
    "m 김정) 자주통일 새,?r열1나가야1보",
    "靑$김기식 논j:정면돌파 태세9선관위 '권해석 결과가 A수",
    "-용9 與 최고위원 출마…난 親~민계 당 위해 Kz",
    "지방서 새누-당 ,q 동v 촉구 이어져d일부는 7참 X대",
    "朴대통' 한 北비핵N 우선1칙따라 공조강화해야종1",
    "I기 나누는 1종석 비서실장과*한병i 정무T석",
    "文대통령n청년일자? 0술방망이 없ae하나씩 만들어가E",
    "되돌아본 한0 정-이A2C대 총~ 선거구 가J스로 지각:정",
    "유U 北제재 북한 5월 당대회서 경제`과V과시 [건xU가나",
    "단Z 朴 뇌2> 핵심B퍼즐 나왔다…i성에 최순실 5원-요구",
    "환영사$하는 문 대x령",
    "국정원장q*병우 직보 ?혹 국장,감찰 조사$중속보",
    "與텃밭/부산 Z변0더G주 4곳이{ 앞서",
    "[ '대통령재벌 총수 u공& 면담 경위 수사종b",
    "北0대! 김정*2향한 성경쟁 무대된l릴레이r토론"
]

results = classifier(examples)
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "주요 키워드": [
    "김정은",
    "문재인",
    "북한",
    "반도",
    "대통령",
    "경제",
    "사회",
    "핵무기",
    "미국",
    "전쟁",
    "의원",
    "정치"
  ],
  "공통된 도메인": "정치",
  "도메인 예측 근거": "제목에서 김정은, 문재인, 북한, 대통령 등 정치적 인물과 주제가 자주 등장하며, 경제, 사회, 핵무기 등 관련된 문제들 또한 언급되어 있습니다."
}


In [ ]:
classifier = create_domain_classifier(llm)

examples = [
    "땅 파= 코l나 격리시설 탈출한 외국인 청_서 VS",
    "모멘트 학교 급식U4칸막이lx곡",
    "초등학생들 경북도청 |들`",
    "한9%^6주택브랜드 공작소 론칭",
    "김영란법 때문에E애플 t이폰 초청70못받7 r국 언.",
    "부7교육청 D식종&자 등 7개 0. 육공무직 19}o 8발",
    "3주 사회서비스F 1급 사무/장 등 u부 직원 q명 공q",
    "코리아^레길 민간추진협의회 2<일 출u식",
    ")보통신산업진흥원 ~#7후보 3.수 압축",
    "0용 공고 보는XK직자들",
    "영상 -랑V 유학 포기할까요…한국 유학9G &6금 n상에 충격",
    "신#계A선호텔 국립전통예술고f*생에3장학금 R원",
    ",시y 9z방송편}인협회5김동연 부총리 초청 토론회",
    "금감원 이어 공정위까지…6부기관[사칭B악h메1 기승",
    "확진 속출F부산 사하구B모든 학교 이틀간 RV수업3q7교 대3",
    "충북.e7D년제 대학 @록금 H21만원…전국 평균 웃돌아",
    "黃 단식중단w도 패스트$랙|대치 @속…靑감찰의혹 공L 가열",
    "9살 학대 )동 4층P베란다 난간to3 넘어 목숨 건9맨발 _출",
    "오q지라이프{임원들 6사주y매각;보도는 사실무b",
    "항소심2선고 출석하는 이%택"
]

results = classifier(examples)
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "keywords": [
    "학교",
    "교육청",
    "부산",
    "경북",
    "학생",
    "공무직",
    "단식",
    "동",
    "사회서비스",
    "지역",
    "주택",
    "금융",
    "기업",
    "임원",
    "법",
    "경제",
    "국립전통예술고등학교",
    "유학",
    "민간추진협의회",
    "출식",
    "보도",
    "방송편자협회",
    "토론회",
    "사칭",
    "대학",
    "학원",
    "임금",
    "학교폭력",
    "청소년",
    "건강",
    "국가기관",
    "시정",
    "일제",
    "고용",
    "사회",
    "문화"
  ],
  "domain": "사회",
  "reason": "주요 키워드 분석 결과, 학교, 교육청, 부산, 경북 등 지역 사회 관련 용어와 공무직, 단식, 사회서비스, 지역 주택, 금융 기업 등 사회 문제와 연결된 키워드가 많이 발견됩니다. 또한 청소년, 건강, 국가기관과 같은 키워드도 포함되어 사회 전체에 대한 다양한 측면을 담고 있는 것으로 판단되었습니다."
}


### 특수문자 비율의 threshold를 0.2로 했을 때의 예시

In [16]:
noisy_df = pd.read_csv('basic_noisy.csv')
noisy_df = noisy_df.drop(columns=['Unnamed: 0', 'target_name', 're_text'])

# 각 target별 개수와 비율 구하기
target_counts = noisy_df['target'].value_counts()
target_ratios = noisy_df['target'].value_counts(normalize=True)

# 결과 출력
result_df = pd.DataFrame({
    'Count': target_counts,
    'Ratio': target_ratios
})

print(result_df)

        Count     Ratio
target                 
3          24  0.214286
6          20  0.178571
5          19  0.169643
0          15  0.133929
2          15  0.133929
1          13  0.116071
4           6  0.053571


In [18]:
df_0 = noisy_df[noisy_df['target']==0]
list_0 = df_0['text'].to_list()

In [19]:
classifier = create_domain_classifier(llm)

examples = list_0

results = classifier(examples)
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "keywords": [
    "화천",
    "눈썰매",
    "축제",
    "연극",
    "산",
    "호우",
    "경보",
    "목전",
    "롯데백화점",
    "예지",
    "독립 선언문"
  ],
  "domain": "문화",
  "reason": "제공된 기사 제목들을 분석해 보면 축제, 연극, 산과 관련된 내용이 눈에 띄며, 예지와 독립선언문 등 문화적 사건에 대한 기사도 포함되어 있습니다. 따라서 이러한 키워드들의 공통점으로 '문화' 도메인으로 분류했습니다."
}


In [21]:
classifier = create_domain_classifier(llm)

df_1 = noisy_df[noisy_df['target']==1]
list_1 = df_1['text'].to_list()
examples = list_1

results = classifier(examples)
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "keywords": [
    "농구",
    "야구",
    "축구",
    "프로야구",
    "대표팀",
    "한국",
    "개막",
    "선수",
    "경기",
    "승리",
    "시즌"
  ],
  "domain": "스포츠",
  "reason": "제공된 기사 제목들은 대부분 농구, 야구, 축구 등 다양한 스포츠 분야의 경기 개막, 선수, 시합 결과 등을 다루고 있습니다. 따라서 이들 기사들의 공통적인 주제는 '스포츠'입니다."
}


In [22]:
classifier = create_domain_classifier(llm)

df_2 = noisy_df[noisy_df['target']==2]
list_2 = df_2['text'].to_list()
examples = list_2

results = classifier(examples)
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "keywords": [
    "김기식",
    "청년",
    "문재인",
    "朴대통령",
    "북한",
    "최순실",
    "지방선거",
    "국정원",
    "경제",
    "핵심",
    "의외",
    "변동",
    "고위원",
    "수사",
    "토론"
  ],
  "domain": "정치",
  "reason": "기사 제목에서 다양한 정치적 키워드가 나타납니다. 대표적으로 '김기식 논란', '朴대통령', '문재인', '지방선거', '북한', '최순실', '국정원' 등이 언급되어 있습니다. 또한, '수사', '토론', '의외', '변동' 등의 키워드는 정치 현상에 대한 분석과 논의를 나타냅니다."
}


In [23]:
classifier = create_domain_classifier(llm)

df_3 = noisy_df[noisy_df['target']==3]
list_3 = df_3['text'].to_list()
examples = list_3

results = classifier(examples)
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "keywords": [
    "학교",
    "청소년",
    "주택",
    "교육",
    "정부",
    "여행",
    "사회",
    "금융",
    "사업",
    "뉴스",
    "인구",
    "학생",
    "경제",
    "법률",
    "오보",
    "동네",
    "공무원"
  ],
  "domain": "사회",
  "reason": "기사 제목에서 다양한 사회 현상에 대한 뉴스와 정보를 확인할 수 있습니다. 학교, 교육, 정부 정책, 경제, 법률, 사회 문제 등이 포함되어 있어 사회 분야를 대표하는 도메인으로 판단됩니다."
}


In [25]:
classifier = create_domain_classifier(llm)

df_5 = noisy_df[noisy_df['target']==5]
list_5 = df_5['text'].to_list()
examples = list_5

results = classifier(examples)
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "keywords": [
    "기업",
    "경제",
    "중소기업",
    "조합",
    "공동주택",
    "건설",
    "인건비",
    "증시",
    "가격지수",
    "해운",
    "금융",
    "자산",
    "생산",
    "제조업",
    "합병",
    "선임",
    "기본계약",
    "취득",
    "도시",
    "부동산",
    "출국금지",
    "브랜드"
  ],
  "domain": "경제",
  "reason": "제공된 기사 제목에서 '중소기업', '증시', '가격지수', '인건비', '합병', '생산', '금융', '부동산' 등 경제 분야와 직접적인 관련이 있는 키워드들이 주로 나타납니다. 또한, '해운', '경기', '중국'과 같은 주요 산업의 상황을 보여주는 단어들도 포함되어 있어 경제 전체를 다루고 있음을 시사합니다."
}


In [26]:
classifier = create_domain_classifier(llm)

df_6 = noisy_df[noisy_df['target']==6]
list_6 = df_6['text'].to_list()
examples = list_6

results = classifier(examples)
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "주요 키워드": [
    "중국",
    "일본",
    "북한",
    "프랑스",
    "미국",
    "아랍",
    "수단",
    "터키",
    "필리핀",
    "영국",
    "유엔",
    "브렉시트",
    "시위",
    "군부",
    "정치",
    "외교",
    "핵",
    "폭발",
    "사망",
    "전쟁",
    "경제",
    "투자",
    "스포츠",
    "교육"
  ],
  "도메인": "세계",
  "근거": "기사 제목에서 다양한 국가, 정치적 사건, 경제 이슈, 국제 관계 등 세계적인 주제들이 나타나고 있습니다."
}


### 특수문자 비율의 threshold를 0.25로 했을 때, target 4의 예시

In [27]:
noisy_df = pd.read_csv('basic_noisy_25.csv')
noisy_df = noisy_df.drop(columns=['Unnamed: 0', 'target_name', 're_text'])

# 각 target별 개수와 비율 구하기
target_counts = noisy_df['target'].value_counts()
target_ratios = noisy_df['target'].value_counts(normalize=True)

# 결과 출력
result_df = pd.DataFrame({
    'Count': target_counts,
    'Ratio': target_ratios
})

print(result_df)

        Count     Ratio
target                 
3          50  0.187266
6          46  0.172285
2          43  0.161049
5          39  0.146067
0          37  0.138577
1          30  0.112360
4          22  0.082397


In [28]:
classifier = create_domain_classifier(llm)

df_4 = noisy_df[noisy_df['target']==4]
list_4 = df_4['text'].to_list()
examples = list_4

results = classifier(examples)
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "keywords": [
    "스마트폰",
    "모바일",
    "로봇",
    "아이폰",
    "개발자",
    "게임",
    "기술",
    "인공지능",
    "디지털",
    "시큐리티",
    "전자파",
    "화이트 그래핀",
    "콘텐츠",
    "5G",
    "반도체",
    "수급",
    "패널",
    "고속인터넷",
    "목성",
    "챗봇",
    "데이터"
  ],
  "domain": "기술",
  "reason": "제공된 기사 제목들은 스마트폰, 로봇, 인공지능, 게임, 반도체 등 기술 분야를 다루고 있습니다. 또한 디지털 콘텐츠, 시큐리티, 전자파, 5G와 같은 키워드가 나타나 기술 발전과 관련된 주제들을 보여줍니다."
}


### 도메인이 예측한 {target: 도메인}

In [5]:
domains = ['문화', '스포츠', '정치', '사회', '기술', '경제', '세계']

target_to_domain = {
    '문화': 0,
    '스포츠': 1,
    '정치': 2,
    '사회': 3,
    '기술': 4,
    '경제': 5,
    '세계': 6
}

## LLM 키워드 뽑기

In [6]:
noisy_df = pd.read_csv('../train_noisy.csv')
noisy_df = noisy_df.drop(columns=['target_name', 're_text'])
noisy_df

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2
3,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6
4,ynat-v1_train_00006,프로야구~롯TKIAs광주 경기 y천취소,1
...,...,...,...
1582,ynat-v1_train_02787,13이 노바 라이n2·미J어패0 T3 10 결G상품% *,6
1583,ynat-v1_train_02788,남원소식 춘>X학%단+장Rn 모집,0
1584,ynat-v1_train_02789,"이총리,세4. H직 완fl지 못해E할 일 꽤;남아",2
1585,ynat-v1_train_02792,"높`X#E율…}BO Q""[/선수 몸값 상승 CAO",1


In [7]:
# 각 target별 개수와 비율 구하기
target_counts = noisy_df['target'].value_counts()
target_ratios = noisy_df['target'].value_counts(normalize=True)

# 결과 출력
result_df = pd.DataFrame({
    'Count': target_counts,
    'Ratio': target_ratios
})

print(result_df)

        Count     Ratio
target                 
6         229  0.144297
2         228  0.143667
5         227  0.143037
3         226  0.142407
4         226  0.142407
0         226  0.142407
1         225  0.141777


In [8]:
domain_to_target = {v: k for k, v in target_to_domain.items()}

# noisy_df의 target 열을 텍스트로 변환
noisy_df['target'] = noisy_df['target'].map(domain_to_target)
noisy_df

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,기술
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,사회
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",정치
3,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,세계
4,ynat-v1_train_00006,프로야구~롯TKIAs광주 경기 y천취소,스포츠
...,...,...,...
1582,ynat-v1_train_02787,13이 노바 라이n2·미J어패0 T3 10 결G상품% *,세계
1583,ynat-v1_train_02788,남원소식 춘>X학%단+장Rn 모집,문화
1584,ynat-v1_train_02789,"이총리,세4. H직 완fl지 못해E할 일 꽤;남아",정치
1585,ynat-v1_train_02792,"높`X#E율…}BO Q""[/선수 몸값 상승 CAO",스포츠


In [9]:
import pandas as pd
from collections import defaultdict
from typing import Dict, List
import json
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

def create_domain_keyword_extractor(llm):
    """도메인별 키워드 추출기 생성"""
    
    # JSON 출력 파서 초기화
    parser = JsonOutputParser()
    
    # 프롬프트 템플릿 설정
    prompt = ChatPromptTemplate.from_messages([
        ("system", 
         '''당신은 여러 텍스트를 분석하여 해당 도메인을 대표하는 핵심 키워드를 추출하는 전문가입니다.
         주어진 텍스트들은 모두 같은 도메인에 속하며, 텍스트에 노이즈가 첨가되어있습니다.
         이 텍스트들을 분석하여 다음을 만족하는 키워드를 추출해주세요.
         
         1. 해당 도메인을 가장 잘 나타내는 핵심 키워드 50개를 추출해주세요
         2. 키워드는 해당 도메인을 다른 도메인과 구분 짓는 특징적인 단어여야 합니다
         3. 일반적인 단어보다는 도메인 특화된 전문 용어나 고유명사를 선호합니다
         4. 추출된 키워드는 중요도 순으로 정렬해주세요
         
         현재 분석 중인 도메인: {domain}
         '''),
        ("human", 
         '''다음 텍스트들을 분석하여 도메인 특화된 키워드를 추출해주세요:
         
         텍스트들: {examples}
         
         {format_instructions}
         ''')
    ])
    
    # 체인 생성
    chain = prompt | llm | parser
    
    def extract_keywords(domain: str, texts: List[str]) -> List[str]:
        """특정 도메인의 텍스트들에서 키워드 추출"""
        response = chain.invoke({
            "domain": domain,
            "examples": "\n".join(texts),
            "format_instructions": """출력은 다음 JSON 형식이어야 합니다:
            {
                "keywords": ["키워드1", "키워드2", ...]
            }"""
        })
        return response["keywords"]
    
    return extract_keywords

def process_domains_and_save(noisy_df: pd.DataFrame, llm, output_path: str = "domain_keywords.csv"):
    """모든 도메인의 텍스트를 처리하고 결과를 CSV로 저장"""
    
    # 키워드 추출기 생성
    extractor = create_domain_keyword_extractor(llm)
    
    # 도메인별 키워드 저장할 딕셔너리
    domain_keywords = defaultdict(list)
    
    # 각 도메인별로 처리
    for domain in noisy_df['target'].unique():
        # 해당 도메인의 텍스트만 추출
        domain_texts = noisy_df[noisy_df['target'] == domain]['text'].tolist()
        
        # 키워드 추출
        keywords = extractor(domain, domain_texts)
        
        # 결과 저장
        domain_keywords[domain] = keywords
    
    # 결과를 데이터프레임으로 변환
    # 가장 긴 키워드 리스트 길이 찾기
    max_keywords = max(len(keywords) for keywords in domain_keywords.values())
    
    # 모든 리스트를 같은 길이로 맞추기 (짧은 리스트는 None으로 패딩)
    for domain in domain_keywords:
        domain_keywords[domain].extend([None] * (max_keywords - len(domain_keywords[domain])))
    
    # 데이터프레임 생성 및 저장
    result_df = pd.DataFrame(domain_keywords)
    result_df.to_csv(output_path, index=False, encoding='utf-8')
    
    return result_df

In [39]:
# 결과 생성 및 저장
result_df = process_domains_and_save(noisy_df, llm, "domain_keywords_word.csv")

# 결과 확인
print(result_df.head())

            기술      사회    정치               세계   스포츠     경제      문화
0         삼성전자    노동실태  국정원장      중국 해외투자 안전도  프로농구     경기      만화
1  5GX MES 플랫폼     교육부  朴대통령     3중 대사관 조기 게양    축구    부동산   독립선언문
2         스마트폰  어린이 안전    북한    한아세 베트남 관계 악화   김재현     금리     사진전
3        디스플레이    한국노총   김정은  페루 대통령 정치 공정 제안   손흥민     외환   사진미술관
4         지문인식    사회복지   무수성        이스라엘 정보당국   류현진  인플레이션  전국리기대회


### 텍스트마다 키워드 뽑고, 중복 제거하기

In [10]:
import pandas as pd
from typing import Dict, List, Set
import json
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

def create_text_keyword_extractor(llm):
    """개별 텍스트에서 키워드를 추출하는 함수 생성"""
    
    # JSON 출력 파서 초기화
    parser = JsonOutputParser()
    
    # 프롬프트 템플릿 설정
    prompt = ChatPromptTemplate.from_messages([
        ("system", 
         '''당신은 텍스트를 분석하여 가장 중요한 키워드를 추출하는 전문가입니다.
         주어진 텍스트는 {target} 도메인에 속합니다. 이 텍스트를 분석하여:
         
         1. 텍스트의 핵심 내용을 가장 잘 나타내는 1-2개의 키워드만 추출해주세요
         2. 키워드는 해당 도메인을 특징짓는 전문 용어나 고유명사여야 합니다
         3. 일반적인 단어는 제외하고, 도메인 특화된 단어만 선택해주세요
         4. 적절한 키워드가 없다면 빈 리스트를 반환해도 됩니다
         '''),
        ("human", 
         '''다음 텍스트에서 {target} 도메인의 핵심 키워드를 1-2개만 추출해주세요:
         
         텍스트: {text}
         
         {format_instructions}
         ''')
    ])
    
    chain = prompt | llm | parser
    
    def extract_keywords(target: str, text: str) -> List[str]:
        """단일 텍스트에서 키워드 추출"""
        response = chain.invoke({
            "target": target,
            "text": text,
            "format_instructions": """출력은 다음 JSON 형식이어야 합니다:
            {
                "keywords": ["키워드1"] 또는 ["키워드1", "키워드2"] 또는 []
            }"""
        })
        return response["keywords"]
    
    return extract_keywords

def process_target_texts(df_target: pd.DataFrame, target: str, llm) -> Dict:
    """특정 타겟의 모든 텍스트를 처리하고 결과 저장"""
    
    # 키워드 추출기 생성
    extractor = create_text_keyword_extractor(llm)
    
    # 모든 키워드를 저장할 세트 (중복 자동 제거)
    all_keywords = set()
    
    # 각 텍스트별로 처리
    for idx, text in enumerate(df_target['text']):
        try:
            # 키워드 추출
            keywords = extractor(target, text)
            # 추출된 키워드가 있으면 세트에 추가
            all_keywords.update(keywords)
            
            # 진행상황 출력 (선택사항)
            if (idx + 1) % 10 == 0:
                print(f"{target} 도메인: {idx + 1}/{len(df_target)} 텍스트 처리 완료")
                
        except Exception as e:
            print(f"텍스트 처리 중 오류 발생: {e}")
            continue
    
    # 결과 딕셔너리 생성
    result = {
        "target": target,
        "keywords": list(all_keywords)  # 중복이 제거된 키워드 리스트
    }
    
    return result

def save_keywords(result: Dict, output_dir: str = "keywords"):
    """추출된 키워드를 JSON 파일로 저장"""
    import os
    
    # 저장 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)
    
    # JSON 파일로 저장
    target = result["target"]
    output_path = os.path.join(output_dir, f"{target}_keywords.json")
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)

def combine_all_keywords(output_dir: str, output_path: str = "all_keywords.csv"):
    """저장된 모든 JSON 파일을 읽어서 하나의 DataFrame으로 통합"""
    import os
    import glob
    
    # 모든 JSON 파일 읽기
    json_files = glob.glob(os.path.join(output_dir, "*_keywords.json"))
    
    # 결과를 저장할 딕셔너리
    all_keywords = {}
    
    # 각 JSON 파일 처리
    for json_file in json_files:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            all_keywords[data['target']] = data['keywords']
    
    # DataFrame 생성
    # 가장 긴 키워드 리스트 찾기
    max_len = max(len(keywords) for keywords in all_keywords.values())
    
    # 길이 맞추기
    for target in all_keywords:
        all_keywords[target].extend([None] * (max_len - len(all_keywords[target])))
    
    # DataFrame 생성 및 저장
    df = pd.DataFrame(all_keywords)
    df.to_csv(output_path, index=False, encoding='utf-8')
    
    return df

In [12]:
# 각 타겟별로 처리
targets = ['문화', '스포츠', '정치', '사회', '기술', '경제', '세계']
for target in targets:
    # 타겟별 데이터 추출
    df_target = noisy_df[noisy_df['target'] == target]
    
    # 키워드 추출 및 중복 제거
    result = process_target_texts(df_target, target, llm)
    
    # 결과 저장
    save_keywords(result)
    print(f"\n{target} 도메인 처리 완료")
    print(f"추출된 고유 키워드 수: {len(result['keywords'])}")

# 모든 결과 통합
final_df = combine_all_keywords('keywords', 'all_domain_keywords.csv')
print(final_df.head())

문화 도메인: 10/226 텍스트 처리 완료
문화 도메인: 20/226 텍스트 처리 완료
문화 도메인: 30/226 텍스트 처리 완료
문화 도메인: 40/226 텍스트 처리 완료
문화 도메인: 50/226 텍스트 처리 완료
문화 도메인: 60/226 텍스트 처리 완료
문화 도메인: 70/226 텍스트 처리 완료
문화 도메인: 80/226 텍스트 처리 완료
문화 도메인: 90/226 텍스트 처리 완료
문화 도메인: 100/226 텍스트 처리 완료
문화 도메인: 110/226 텍스트 처리 완료
문화 도메인: 120/226 텍스트 처리 완료
문화 도메인: 130/226 텍스트 처리 완료
문화 도메인: 140/226 텍스트 처리 완료
문화 도메인: 150/226 텍스트 처리 완료
문화 도메인: 160/226 텍스트 처리 완료
텍스트 처리 중 오류 발생: Invalid json output: {
    "keywords": []
} 


주어진 텍스트는 '명말청6 7기[?국7사mz들은 3IUSb아남@p' 와 같이 의미를 파악할 수 없는 형태로 제시되어 있습니다. 따라서 문화 도메인의 키워드를 추출하기 어렵습니다.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE
문화 도메인: 170/226 텍스트 처리 완료
문화 도메인: 180/226 텍스트 처리 완료
문화 도메인: 190/226 텍스트 처리 완료
문화 도메인: 200/226 텍스트 처리 완료
문화 도메인: 210/226 텍스트 처리 완료
문화 도메인: 220/226 텍스트 처리 완료
텍스트 처리 중 오류 발생: 'NoneType' object is not subscriptable

문화 도메인 처리 완료
추출된 고유 키워드 수: 70
스포츠 도메인: 10/225 텍스트 처리 완료
스포츠 도메인: 20/225 텍스트 처리 완료
스포츠 도메인: 30/225 텍스트 처리 완료
스포

In [13]:
final_df

,기술,사회,문화,스포츠,경제,세계,정치
0,py체,대외고 통학차,스포X카,득점왕,기업수익,대선,국회
1,뉴욕 필름 페스,미세먼지,교향곡,MLB,배당,협약 이행,책 추진
2,아이펀팩`리,교황청,포퓰리즘,올림픽,경기침체,해상N력,소통 강화
3,왓x,항소심,불교,바셀로나,합병,정상회의,국회사무처
4,AI 업지능,파장,모z르트w,바르사 빌바오,부품주,美4中Z군사적 충돌,올림픽
...,...,...,...,...,...,...,...
139,None,거제 E0 출입제한,None,도전,None,None,None
140,None,지방자치단체,None,두산,None,None,None
141,None,D식종&자,None,프로야구,None,None,None
142,None,민간추진협의회,None,루카쿠,None,None,None


## Noisy text 복원

In [14]:
import pandas as pd
from typing import Dict, List
import json
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

def create_text_restorer(llm):
    """텍스트 복원기 생성"""
    
    # JSON 출력 파서 초기화
    parser = JsonOutputParser()
    
    # 프롬프트 템플릿 설정
    prompt = ChatPromptTemplate.from_messages([
        ("system", 
         '''당신은 노이즈가 낀 텍스트를 원래의 뉴스 기사 제목으로 복원하는 전문가입니다.
         주어진 텍스트는 {target} 도메인의 뉴스 기사 제목이었으나 노이즈가 끼었습니다.
         
         복원 규칙:
         1. 노이즈가 낀 텍스트를 보고 원래 의미를 파악할 수 있다면, 해당 의미를 살려 복원해주세요
         2. 텍스트가 너무 손상되어 의미 파악이 어렵다면, 제공된 키워드 리스트에서 적절한 키워드를 선택해 
            {target} 도메인에 맞는 새로운 뉴스 기사 제목을 생성해주세요
         3. 복원된 텍스트의 길이는 원본 텍스트의 글자 수와 최대한 비슷하게 맞춰주세요
            (원본 길이: {original_length}자)
         4. 복원 방식(복원/생성)을 명시해주세요
         '''),
        ("human", 
         '''다음 정보를 바탕으로 텍스트를 복원해주세요:
         
         도메인: {target}
         노이즈가 낀 텍스트: {noisy_text}
         참고 키워드 리스트: {keywords}
         
         {format_instructions}
         ''')
    ])
    
    chain = prompt | llm | parser
    
    def restore_text(target: str, noisy_text: str, keywords: List[str]) -> Dict:
        """텍스트 복원 실행"""
        response = chain.invoke({
            "target": target,
            "noisy_text": noisy_text,
            "keywords": ", ".join(keywords),
            "original_length": len(noisy_text),
            "format_instructions": """출력은 다음 JSON 형식이어야 합니다:
            {
                "restored_text": "복원된 텍스트",
                "method": "복원" 또는 "생성"
            }"""
        })
        return response
    
    return restore_text

def process_dataframe(noisy_df: pd.DataFrame, keyword_df: pd.DataFrame, llm) -> pd.DataFrame:
    """전체 데이터프레임 처리"""
    
    # 결과를 저장할 새로운 데이터프레임 생성
    result_df = noisy_df.copy()
    result_df['re_text'] = ''
    result_df['restoration_method'] = ''
    
    # 텍스트 복원기 생성
    restorer = create_text_restorer(llm)
    
    # 각 행별로 처리
    for idx, row in result_df.iterrows():
        try:
            # 해당 타겟의 키워드 리스트 가져오기
            target_keywords = keyword_df[row['target']].dropna().tolist()
            
            # 텍스트 복원
            result = restorer(
                target=row['target'],
                noisy_text=row['text'],
                keywords=target_keywords
            )
            
            # 결과 저장
            result_df.at[idx, 're_text'] = result['restored_text']
            result_df.at[idx, 'restoration_method'] = result['method']
            
            # 진행상황 출력 (100개마다)
            if (idx + 1) % 100 == 0:
                print(f"{idx + 1}/{len(result_df)} 처리 완료")
                
        except Exception as e:
            print(f"ID {row['ID']} 처리 중 오류 발생: {e}")
            result_df.at[idx, 're_text'] = row['text']  # 오류 시 원본 텍스트 유지
            result_df.at[idx, 'restoration_method'] = 'error'
            continue
    
    return result_df

def save_results(df: pd.DataFrame, output_path: str = "restored_texts.csv"):
    """결과를 CSV 파일로 저장"""
    df.to_csv(output_path, index=False, encoding='utf-8')

In [15]:
# 전체 데이터프레임 처리
restored_df = process_dataframe(noisy_df, final_df, llm)

# 결과 저장
save_results(restored_df, "restored_news_titles.csv")

# 결과 확인
print("\n복원 방법 통계:")
print(restored_df['restoration_method'].value_counts())

# 샘플 결과 확인
print("\n복원 예시:")
samples = restored_df.sample(5)
for _, row in samples.iterrows():
    print(f"\n원본: {row['text']}")
    print(f"복원: {row['re_text']}")
    print(f"방법: {row['restoration_method']}")

ID ynat-v1_train_00046 처리 중 오류 발생: Invalid json output: ```json
{
"restored_text": "5자농 R4은행 삼[m명 꺾직",
"method": "복원"
}
``` 


**해설:**

주어진 노이즈가 낀 텍스트는 스포츠 관련 뉴스 제목 형태로 보이며, 일부 단어들이 잘못 표기되어 있습니다.  '5자농 R4은행 삼[m명 꺾직' 을 통해 '5자농 R4은행 삼성 명 꺾직'으로 복원할 수 있을 것으로 추정됩니다.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE
100/1587 처리 완료
200/1587 처리 완료
ID ynat-v1_train_00474 처리 중 오류 발생: 'NoneType' object is not subscriptable
300/1587 처리 완료
ID ynat-v1_train_00566 처리 중 오류 발생: Invalid json output: ```json
{
"restored_text": "한국 교향악단 오케스트라 공연",
"method": "복원"
}
``` 



**설명:**

노이즈가 끼어있는 ' [간 @기와 ;외X 음k5quA타 3q' 에서  '교향악단', '오케스트라' 와 같은 키워드를 추출하여 원래 의미로 가정하여 복원했습니다.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE
ID ynat-v1_train_00579 처리 중 오류 발생: Invalid json output: ```json
{
"restored_text": "영국, 레지스탕스 혼란에 C z?獨(佛…*y한 입장 제Q해"[합",
"method": "복원"
}
``` 


**요약:**

제공된 노이즈가 낀 텍스트는  '영국의

### 데이터 증강

In [33]:
import pandas as pd
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# 요청 사항 설정
req = "도메인을 명확하게 나타내는 뉴스 기사 제목을, 30글자가 넘지 않도록 도메인별로 30개씩 생성해 주세요."

# JSONOutputParser 초기화
parser = JsonOutputParser()

# 도메인별 요구사항
domain_requirements = """
- 문화: 건강, 자동차 리뷰, 교통 상황, 여행지 추천, 맛집 탐방, 패션 트렌드, 공연 정보, 신간 도서, 만화/웹툰, 학술/문화재, 종교 행사, 기상 소식 등의 주제를 포함해야 하며, 제목은 생활 속 유용한 정보나 최신 문화 트렌드를 반영해야 합니다. 특정 활동이나 장소가 포함되면 더 좋습니다.
- 스포츠: 야구, 축구, 농구, 배구, 골프와 관련된 스포츠 소식이나 팀 이름, 선수 이름, 경기 결과를 포함하면 좋습니다. 제목은 해당 종목의 주요 경기 소식, 팀이나 선수의 승패, 리그 경기 일정을 반영하는 내용이어야 합니다. 팬들에게 흥미를 줄 수 있도록 특정 팀, 선수, 경기 기록이 포함되면 좋습니다.
- 정치: 대통령실, 국회, 정당 정책, 외교 관계, 국방 관련 주제로 정치적인 이슈를 다루면 좋습니다. 사회 분야와 헷갈리지 않도록 법안, 외교 협력, 정부 정책 등의 정치적 의제를 명확히 드러내고, 국내외 정치적 흐름이나 중요한 정부 발표가 포함된 제목이어야 합니다.
- 사회: 사건/사고, 법원 판결, 교육 정책, 복지 서비스, 노동 문제, 환경 이슈, 여성과 아동 문제, 다문화 관련 주제를 다루면 좋습니다. 정치나 문화 분야와 헷갈리지 않도록, 사회적인 현안이나 공공의 문제를 강조하고, 특히 법원이나 검찰 관련 사안, 주요 사건을 포함한 제목이 좋습니다.
- 기술: 유명 IT 기업 이름이나 신기술 관련 용어가 포함되어 기술의 발전과 변화를 잘 드러내는 제목을 만들어주세요. 최신 과학 기술, IT 산업의 발전, 기술 기업의 신제품 발표, 혁신적인 기술 트렌드와 관련되면 좋습니다. 기술의 응용이나 영향을 언급하면 더욱 좋습니다.
- 경제: 경제 정책, 금융 시장, 부동산, 취업/창업, 소비자 관련 정보를 담기면 좋습니다. 주식, 펀드, 금리, 부동산 가격 상승/하락 등 경제 변동을 보여주는 내용을 포함하고, 특히 투자나 기업의 경제적 성장, 경기 흐름을 반영하는 제목이 좋습니다.
- 세계: 미국, 중국, 일본, 유럽 등 국가 이름이나 국제기구와 관련된 글로벌 소식을 담기면 좋습니다. 세계적인 정세 변화를 반영하고, 국가 간 관계, 국제 협력, 글로벌 이슈와 같은 주제를 다루며, 특정 지역이나 국가의 중요 소식이 포함된 제목을 만들어주세요.
"""

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages([
    ("system",
     '''당신은 특정 도메인의 기사 제목을 생성하는 어시스턴트입니다.
     다음 조건을 기반으로 기사 제목을 생성해 주세요:

     - 주어진 도메인 "{{target}}"을 명확하게 나타내는 한문장 내외의 적절한 뉴스 기사 제목을 30개 생성하세요.
     - 제목은 반드시 JSON 배열 형식으로 반환하고, 도메인을 "target" 키로, 생성한 제목을 "text" 키로 표시합니다.

     도메인별 요구사항:
     {domain_requirements}

     주의: 응답은 JSON 배열 형식으로만 반환해야 하며, 그 외의 텍스트는 포함하지 마세요. 
     반드시 JSON 형식으로만 반환하고, 배열의 각 항목은 {{"target": "{{target}}", "text": "<생성한 제목>"}} 형식이어야 합니다.'''),
    ("user",
     "#Format: {format_instructions}\n\n#Requirements: {req}\n\n#Domain: {target}")
])

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | llm | parser

def generate_domain_titles(domains: List[str]) -> pd.DataFrame:
    """
    여러 도메인에 대한 뉴스 제목을 생성하고 DataFrame으로 반환하는 함수
    
    Args:
        domains (List[str]): 뉴스 도메인 리스트
    
    Returns:
        pd.DataFrame: target(도메인)과 text(제목) 컬럼을 가진 DataFrame
    """
    data = []
    
    for domain in domains:
        try:
            print(f"\n{domain} 도메인의 뉴스 제목 생성 중...")
            response = chain.invoke({
                "req": req,
                "target": domain,
                "domain_requirements": domain_requirements,
                "format_instructions": parser.get_format_instructions()
            })
            
            # response가 None인지 확인
            if response is None:
                print("오류 발생: chain.invoke()에서 None이 반환되었습니다.")
            elif isinstance(response, list):  
                data.extend(response)
            else:
                print("올바르지 않은 JSON 응답:", response)

        except Exception as e:
            print(f"{domain} 도메인에서 예외 발생: {str(e)}")

    return pd.DataFrame(data)

In [ ]:
domains = ['문화', '스포츠', '정치', '사회', '기술', '경제', '세계']

# 뉴스 제목 생성
df = generate_domain_titles(domains)

In [ ]:
target_to_domain = {
    '문화': 0,
    '스포츠': 1,
    '정치': 2,
    '사회': 3,
    '기술': 4,
    '경제': 5,
    '세계': 6
}

df['target'] = df['target'].map(target_to_domain)

In [ ]:
df.to_csv('news_titles.csv')

In [ ]:
restored_df = restored_df.drop(columns=['text'])
restored_df = restored_df.rename(columns={'re_text': 'text'})
restored_df['target'] = restored_df['target'].map(target_to_domain)
restored_df

,ID,target,text,restoration_method
0,ynat-v1_train_00000,4,"KT AI 서비스 이용기간 2년 단축, Q분종 U2보",복원
1,ynat-v1_train_00001,3,K찰국 회장 노조 통합협상,복원
2,ynat-v1_train_00002,2,"김정은, 북미 대화 정면돌파 필요하나?",복원
3,ynat-v1_train_00004,6,미대선 앞두고 R2프랑스 단 발 비해 감시 강화,복원
4,ynat-v1_train_00006,1,프로야구 롯데 KIA 광주 경기 취소,복원
...,...,...,...,...
1582,ynat-v1_train_02787,6,13일 노바 라이2·미군 합동演習 이루어진다.,복원
1583,ynat-v1_train_02788,0,남원소식춘향단 모집,복원
1584,ynat-v1_train_02789,2,"이총리, 국회 직 무기대상책 추진해야",복원
1585,ynat-v1_train_02792,1,"높`X#E율…}BO Q""[/선수 몸값 상승 CAO",error


In [134]:
restored_df = restored_df[restored_df['restoration_method']!='error']

In [ ]:
gemma_made = pd.concat([df, restored_df], ignore_index=True)
gemma_made

,target,text,ID,restoration_method
0,0,"'나의 아티스트, 나만의 색깔' 서울 시립 미술관 신규 전시 기대감 UP",NaN,NaN
1,0,여름 맞이 '냉면 vs 비빔밥' 맛집 대결 당신은 어떤 선택,NaN,NaN
2,0,"'스타일 변신', 패션 트렌드 집중 분석 2023년 여름 인기 소품 대방출",NaN,NaN
3,0,"'인생영화, 다시 한번' 오리지널 OST 반복 감상 이유",NaN,NaN
4,0,독서 습관 형성 새롭게 출간된 베스트셀러 도서 추천 목록,NaN,NaN
...,...,...,...,...
2390,3,텅 빈 교실 선생님 외롭게,ynat-v1_train_02786,복원
2391,6,13일 노바 라이2·미군 합동演習 이루어진다.,ynat-v1_train_02787,복원
2392,0,남원소식춘향단 모집,ynat-v1_train_02788,복원
2393,2,"이총리, 국회 직 무기대상책 추진해야",ynat-v1_train_02789,복원


In [136]:
gemma_made.to_csv('gemma_made.csv')